# **Workshop #1 - Notebook 3: Data Transformation**

In this notebook, various data analysis tasks are performed, such as data cleansing and transformation. For example, the transformation of several columns is executed here using Pandas, through a module called *connection.py* that establishes the connection to a PostgreSQL database that stores the raw data. Later in this notebook, the loading of the clean data is going to be stored in a new table called “candidates_hired”.

---

### **Setting the environment**

In [1]:
import os 
print(os.getcwd())

try:
    os.chdir("../../Workshop #1")
except FileNotFoundError:
    print("""
        FileNotFoundError - Puede que el directorio no exista o no estés en la ruta especificada.
        Si ya te funcionó, no ejecutes este bloque de nuevo, ya que el directorio actual es Workshop #1.
        
        Confirma que la ruta en donde te ubicas es la correcta con la siguiente dirección:
        """)

print(os.getcwd())

c:\Users\marti\OneDrive\Escritorio - PC\Ingenieria de Datos e IA - UAO\Semestre 4\ETL\Semana #1 - #6\Workshop #1\notebooks
c:\Users\marti\OneDrive\Escritorio - PC\Ingenieria de Datos e IA - UAO\Semestre 4\ETL\Semana #1 - #6\Workshop #1


### **Importing libraries and modules**

In [2]:
import pandas as pd

from functions.db_connection.connection import creating_engine

### **Creating the engine**

In [3]:
engine = creating_engine()

### **Loading the dataset**

In [4]:
df = pd.read_sql_table("candidates_raw", engine, parse_dates=["Application Date"])
df

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Security,2,1
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Game Development,1,2
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,System Administration,6,1
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Database Administration,0,0


## ***Data Transformation***

### **Renaming the columns**

In [5]:
renamed_columns = {
    'First Name': 'first_name',
    'Last Name': 'last_name',
    'Email': 'email',
    'Application Date': 'application_date',
    'Country': 'country',
    'YOE': 'yoe',
    'Seniority': 'seniority',
    'Technology': 'technology',
    'Code Challenge Score': 'code_challenge_score',
    'Technical Interview Score': 'technical_interview_score'
}

df = df.rename(columns=renamed_columns)
df.columns

Index(['first_name', 'last_name', 'email', 'application_date', 'country',
       'yoe', 'seniority', 'technology', 'code_challenge_score',
       'technical_interview_score'],
      dtype='object')

### **Grouping *"technology"* by category**

In [6]:
df["technology"].unique()

array(['Data Engineer', 'Client Success', 'QA Manual',
       'Social Media Community Management', 'Adobe Experience Manager',
       'Sales', 'Mulesoft', 'DevOps', 'Development - CMS Backend',
       'Salesforce', 'System Administration', 'Security',
       'Game Development', 'Development - CMS Frontend',
       'Security Compliance', 'Development - Backend', 'Design',
       'Business Analytics / Project Management',
       'Development - Frontend', 'Development - FullStack',
       'Business Intelligence', 'Database Administration',
       'QA Automation', 'Technical Writing'], dtype=object)

In [7]:
renamed_technologies = {
    'Development - Backend': 'Development and Programming',
    'Development - Frontend': 'Development and Programming',
    'Development - FullStack': 'Development and Programming',
    'Development - CMS Backend': 'Development and Programming',
    'Development - CMS Frontend': 'Development and Programming',
    'Game Development': 'Development and Programming',
    
    'Business Analytics / Project Management': 'Analytics and Business Intelligence',
    'Business Intelligence': 'Analytics and Business Intelligence',
    
    'QA Manual': 'QA and Testing',
    'QA Automation': 'QA and Testing',
    
    'System Administration': 'Systems Administration and Management',
    'Database Administration': 'Systems Administration and Management',
    'Security': 'Systems Administration and Management',
    'Security Compliance': 'Systems Administration and Management',
    
    'Client Success': 'Project and Client Management',
    'Sales': 'Project and Client Management',
    
    'Design': 'Design and Creativity',
    'Social Media Community Management': 'Design and Creativity',
    
    'DevOps': 'Automation and DevOps',
    'Mulesoft': 'Automation and DevOps',
    'Salesforce': 'Automation and DevOps',
}

In [8]:
df['technology'] = df['technology'].replace(renamed_technologies)

In [9]:
df["technology"].unique()

array(['Data Engineer', 'Project and Client Management', 'QA and Testing',
       'Design and Creativity', 'Adobe Experience Manager',
       'Automation and DevOps', 'Development and Programming',
       'Systems Administration and Management',
       'Analytics and Business Intelligence', 'Technical Writing'],
      dtype=object)

### **Creating an "*is_hired*" column**

This column contains a boolean value that follows the required logical statement that code_challenge_score and technical_interview_score must be greater than or equal to 7.

In [10]:
df['is_hired'] = (df['code_challenge_score'] >= 7) & (df['technical_interview_score'] >= 7)
df

,first_name,last_name,email,application_date,country,yoe,seniority,technology,code_challenge_score,technical_interview_score,is_hired
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3,False
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10,False
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Project and Client Management,10,9,True
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA and Testing,7,1,False
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Design and Creativity,9,7,True
...,...,...,...,...,...,...,...,...,...,...,...
49995,Bethany,Shields,rocky_mitchell@hotmail.com,2022-01-09,Dominican Republic,27,Trainee,Systems Administration and Management,2,1,False
49996,Era,Swaniawski,dolores.roob@hotmail.com,2020-06-02,Morocco,21,Lead,Development and Programming,1,2,False
49997,Martin,Lakin,savanah.stracke@gmail.com,2018-12-15,Uganda,20,Trainee,Systems Administration and Management,6,1,False
49998,Aliya,Abernathy,vivienne.fritsch@yahoo.com,2020-05-30,Czech Republic,20,Senior,Systems Administration and Management,0,0,False


## ***Loading the clean data***

The table is created using the engine and the Pandas `to_sql()` function. The data types are shown below the code.

In [11]:
df.to_sql('candidates_hired', engine, if_exists='replace', index=False)

1000